In [1]:
from google.colab import drive
drive.mount("/dir")

Mounted at /dir


In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

sh = gc.create('GBVS360 Result')

# Open our new sheet and add some data.
worksheet = gc.open('GBVS360 Result').sheet1

In [3]:
#import glob
import numpy as np
import pandas as pd

import PIL
import matplotlib.pyplot as plt

from scipy.ndimage import map_coordinates
import scipy.io
import scipy as sp
import sklearn.preprocessing
from skimage.transform import resize
import numpy.matlib
import math

import cv2
import os,re, glob
from functools import partial

EPSILON = np.finfo('float').eps
%matplotlib inline

### functions

In [4]:
def normalize_metric(x, method='standard', axis=None):
	x = np.array(x, copy=False)
	if axis is not None:
		y = np.rollaxis(x, axis).reshape([x.shape[axis], -1])
		shape = np.ones(len(x.shape))
		shape[axis] = x.shape[axis]
		if method == 'standard':
			res = (x - np.mean(y, axis=1).reshape(shape)) / np.std(y, axis=1).reshape(shape)
		elif method == 'range':
			res = (x - np.min(y, axis=1).reshape(shape)) / (np.max(y, axis=1) - np.min(y, axis=1)).reshape(shape)
		elif method == 'sum':
			res = x / np.float_(np.sum(y, axis=1).reshape(shape))
		else:
			raise ValueError('method not in {"standard", "range", "sum"}')
	else:
		if method == 'standard':
			res = (x - np.mean(x)) / np.std(x)
		elif method == 'range':
			res = (x - np.min(x)) / (np.max(x) - np.min(x))
		elif method == 'sum':
			res = x / float(np.sum(x))
		else:
			raise ValueError('method not in {"standard", "range", "sum"}')
	return res

In [5]:
def KLD(p, q):
	p = normalize_metric(p, method='sum')
	q = normalize_metric(q, method='sum')
	return np.sum(np.where(p != 0, p * np.log((p+EPSILON) / (q+EPSILON)), 0))

In [6]:
def CC(saliency_map1, saliency_map2):
	map1 = np.array(saliency_map1, copy=False)
	map2 = np.array(saliency_map2, copy=False)
	if map1.shape != map2.shape:
		map1 = resize(map1, map2.shape, order=3, mode='constant') 
	map1 = normalize_metric(map1, method='standard')
	map2 = normalize_metric(map2, method='standard')
	return np.corrcoef(map1.ravel(), map2.ravel())[0,1]

In [7]:
def SIM(saliency_map1, saliency_map2):
	map1 = np.array(saliency_map1, copy=False)
	map2 = np.array(saliency_map2, copy=False)
	if map1.shape != map2.shape:
		map1 = resize(map1, map2.shape, order=3, mode='constant') 
	map1 = normalize_metric(map1, method='range')
	map2 = normalize_metric(map2, method='range')
	map1 = normalize_metric(map1, method='sum')
	map2 = normalize_metric(map2, method='sum')
	intersection = np.minimum(map1, map2)
	return np.sum(intersection)

In [8]:
def jsd(p, q, base=np.e):
	return sp.spatial.distance.jensenshannon(p.flatten(),q.flatten())

### __main__

In [9]:
dataset_url = "/dir/My Drive/Design_Credit/Data_Set/GBVS360_SalMap_Output/*.png"
salmap_url = "/dir/My Drive/Design_Credit/Data_Set/Sorted_SalMap/*.png"

In [10]:
col1=[img for img in glob.glob(dataset_url) ]
col2=[map for map in glob.glob(salmap_url) ]
col1.sort()
col2.sort()

In [11]:
def NormalizeData(data):
    return ((data - np.min(data)) / (np.max(data) - np.min(data)))

In [12]:
output=[]
for i in range(85):
  image=np.asarray(PIL.Image.open(col1[i]))
  salMap=np.asarray(PIL.Image.open(col2[i]))

  x1,y1=image.shape
  x2,y2,z2=salMap.shape

  height , width = min(x1,x2), min(y1,y2)
  image=cv2.resize(image, dsize=(width, height))
  salMap=cv2.resize(salMap, dsize=(width, height))
  compare=cv2.cvtColor(salMap,cv2.COLOR_BGR2GRAY)
  
  result = (KLD(image,compare),CC(image,compare),SIM(image,compare),jsd(image,compare))
  output.append(result)

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (128000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (168031112 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (166786848 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (166348800 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (134217728 pixels) exceeds limit of 89478

In [13]:
cell_list = worksheet.range('A1:D85')
i=0
j=0
for cell in cell_list:
  cell.value = output[i][j]
  j+=1
  if(j==4):
    i+=1
    j=0
  
worksheet.update_cells(cell_list)

{'spreadsheetId': '1Lc3lHS1ArxXZ0bn7CokFyervh5VC_t1voWTuQ6UibnM',
 'updatedCells': 340,
 'updatedColumns': 4,
 'updatedRange': 'Sheet1!A1:D85',
 'updatedRows': 85}